In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
import os

In [3]:
img_height, img_width = 224, 224
batch_size = 32
dataset_dir = "/content/drive/MyDrive/dataset-resized"
labels_map = {
    "cardboard": 0,
    "glass": 1,
    "metal": 2,
    "paper": 3,
    "plastic": 4,
    "trash": 5
}

In [4]:
# Load dataset assuming subfolder structure: /class_name/image.jpg
train_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='int',
    validation_split=0.2,
    subset="training",
    seed=123
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='int',
    validation_split=0.2,
    subset="validation",
    seed=123
)

Found 2527 files belonging to 6 classes.
Using 2022 files for training.
Found 2527 files belonging to 6 classes.
Using 505 files for validation.


In [5]:
# Normalize images
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

In [6]:
# Build model using MobileNetV2
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(6, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_ds, validation_data=val_ds, epochs=5)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Epoch 1/5
64/64 ━━━━━━━━━━━━━━━━━━━━ 705s 11s/step - accuracy: 0.5699 - loss: 1.0986 - val_accuracy: 0.8079 - val_loss: 0.5440
Epoch 2/5
64/64 ━━━━━━━━━━━━━━━━━━━━ 8s 129ms/step - accuracy: 0.8400 - loss: 0.4599 - val_accuracy: 0.7683 - val_loss: 0.5812
Epoch 3/5
64/64 ━━━━━━━━━━━━━━━━━━━━ 10s 128ms/step - accuracy: 0.8923 - loss: 0.3223 - val_accuracy: 0.8574 - val_loss: 0.4465
Epoch 4/5
64/64 ━━━━━━━━━━━━━━━━━━━━ 9s 111ms/step - accuracy: 0.9449 - loss: 0.2065 - val_accuracy: 0.8238 - val_loss: 0.5056
Epoch 5/5
64/64 ━━━━━━━━━━━━━━━━━━━━ 8s 129ms/step - accuracy: 0.9584 - loss: 0.1579 - val_accuracy: 0.8396 - val_loss: 0.4799


In [7]:
# Convert to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model
with open('recyclable_classifier.tflite', 'wb') as f:
    f.write(tflite_model)

print("TFLite model created and saved successfully.")

Saved artifact at '/tmp/tmpzi526p42'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_154')
Output Type:
  TensorSpec(shape=(None, 6), dtype=tf.float32, name=None)
Captures:
  133436922998864: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133436922999056: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133433870490256: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133433877926480: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133436922999632: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133433870490640: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133433870492176: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133433870491792: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133436922999440: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133433870492368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1334338704

In [8]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Load one test image (update path as needed)
test_img_path = "/content/drive/MyDrive/dataset-resized/glass/glass50.jpg"
img = image.load_img(test_img_path, target_size=(224, 224))
img_array = image.img_to_array(img) / 255.0
input_data = np.expand_dims(img_array, axis=0).astype(np.float32)

# Load TFLite model
interpreter = tf.lite.Interpreter(model_path="recyclable_classifier.tflite")
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

# Run inference
interpreter.set_tensor(input_index, input_data)
interpreter.invoke()
output = interpreter.get_tensor(output_index)

predicted_class = np.argmax(output)
print("Predicted class index:", predicted_class)


Predicted class index: 1
